In [1]:
import numpy as np
import tensorflow as tf
import tflearn
import random

import nltk

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import json 
import pickle
import warnings
warnings.filterwarnings("ignore")


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
with open('intents.json', encoding="utf8") as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?', ',', '!', '@', '#']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

training = []
output = []
output_empty = [0] * len(classes)

In [5]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

In [6]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])
tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')


In [8]:
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')
# this is training part

Training Step: 1999  | total loss: 0.11339 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.11339 - acc: 0.9984 -- iter: 08/11
Training Step: 2000  | total loss: 0.11325 | time: 0.008s
| Adam | epoch: 1000 | loss: 0.11325 - acc: 0.9985 -- iter: 11/11
--
INFO:tensorflow:c:\Users\harleyquinn\Videos\Chat-Bot-train\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:c:\Users\harleyquinn\Videos\Chat-Bot-train\model.tflearn.data-00000-of-00001
INFO:tensorflow:0
INFO:tensorflow:c:\Users\harleyquinn\Videos\Chat-Bot-train\model.tflearn.index
INFO:tensorflow:0
INFO:tensorflow:c:\Users\harleyquinn\Videos\Chat-Bot-train\model.tflearn.meta
INFO:tensorflow:100


In [9]:


pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

with open('intents.json', encoding="utf8") as json_data:
    intents = json.load(json_data)

model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from c:\Users\harleyquinn\Videos\Chat-Bot-train\model.tflearn


In [10]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [11]:
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("Lets Go!!")

Lets Go!!


In [12]:
def classify(sentence):
    results = model.predict([bow(sentence, words)])[0]
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    return return_list

In [13]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in intents['intents']:
                if i['tag'] == results[0][0]:
                    a = str(random.choice(i['responses']))
                    return a

            results.pop(0)

In [14]:
response("ticket")

'view'

while True:
    input_data= input()
    print(response(input_data))